<a href="https://colab.research.google.com/github/shafayat1004/Data-Mining-Assignments/blob/main/Assignment%203.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# KMeans Clustering on Iris Dataset
## 1910456 Mir Shafayat Ahmed                                                                                                     

In [530]:
!mkdir -p iris
!curl "https://raw.githubusercontent.com/shafayat1004/Data-Mining-Assignments/main/iris/bezdekIris.data" --output iris/bezdekIris.data
divider = "--------------------------------------------------------------------------------------"

A subdirectory or file -p already exists.
Error occurred while processing: -p.
A subdirectory or file iris already exists.
Error occurred while processing: iris.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  4551  100  4551    0     0   5296      0 --:--:-- --:--:-- --:--:--  5298
100  4551  100  4551    0     0   5295      0 --:--:-- --:--:-- --:--:--  5298


In [531]:
import numpy as np
import pandas as pd
import plotly.express as px
pd.set_option('display.max_rows', None)

This is the pseudocode loads data from file and save in the numpy matrix. 
You can also use panda to read the dataset into numpy matrix. data matrix will have data points. Columns have features. Rows have data points  ClassLabel has the labels.

In [532]:
def load(iris_dataset_path):
    column_names     = [
        "Sepal Length",
        "Sepal Width",
        "Petal Length",
        "Petal Width",
        "Class"
    ]
    dataset = \
        pd.read_csv (
            iris_dataset_path,
            names = column_names
        )
    
    data = dataset[[column_names[0], column_names[1], column_names[2], column_names[3]]]
    class_label = dataset[column_names[4]]
    
    return data, class_label

simple centroid initialization function

In [533]:
def initialize_centroids_simple(data, dimension_of_data, no_of_clusters):
    #centroids: [[centroid0:  3 dimensions, , , ]; [centroid1: 3 dimensions ] ... ..]
    centroids = np.zeros((no_of_clusters, dimension_of_data))
    
    random_indexes = np.random.permutation(len(data))                                                # Selecting random data points for each initial Centroid
    for i in range(no_of_clusters):
        centroids[i] = data.iloc[random_indexes[i]]
        
    print(f"Initialized Random Centroids =\n{centroids}")
    print(divider)
    
    return centroids

Centroid initilization using min max

Calculate eucledian distance

In [534]:
def get_euclidean_distance(vector1, vector2):
    difference = vector1 - vector2
    sum_of_squared_diff = np.dot (difference, difference)
    distance = np.sqrt(sum_of_squared_diff)
    
    return distance

In [535]:
def clustering_objective (data, centroids):
    total_sum = float(0)                                                                             # Initial total_sum is 0
    for point in data:                                                                               # Iterating through each datapoint
        total_sum += min([np.dot(point - centroid, point - centroid) for centroid in centroids])     # List comprehension to get a list of all the squared distances between...
                                                                                                     # ...the current datapoint and each of the k centroids. Then using min()...
                                                                                                     # ...to get the minimum distance and adding it to sum.
    return total_sum/float(len(data))                                                                # J is found by dividing sum with the number of datapoints. 1/N * sum

KMeans Function


In [536]:
def kmeans(data, dimension_of_data, no_of_clusters, max_iterations):
    no_of_features = np.size(data, 0)
    
    #centroids: [[centroid0:  , , ,3 dimensions, , , ],  [centroid1: , , ,3 dimensions, , , ],  ... ..]
    centroids = initialize_centroids_simple(data, dimension_of_data, no_of_clusters)
    #cluster_affiliation: cluster_affiliation = [clusternumber, clutsernumber, ..., ..., ..., ...]
    
    #initialize the cluster affiliations. Initially assign -1 
    cluster_affiliation = (np.zeros(len(data)) - 1).astype(int)
    print(f"Initial Cluster Affiliation =\n{cluster_affiliation}")
    print(divider)
    
    data = data.to_numpy()
    
    continue_loop = True
    j_prev = np.inf                                                                                  # So that loop does not break immediately after first Iteration.
    for iteration in range(max_iterations):                                                          # Setting a max limit of loops so that it is not infinite.
        print(f"Iteration {iteration}:")
        for i, point in enumerate(data): #use numpy equivalent code
            
            min_distance = float('inf')
            min_distance_index = None

            #find the closest centroids for each data points
            for cluster_index, centroid in enumerate(centroids): #use numpy equivalent code
                distance = get_euclidean_distance(centroid, point)
                if distance < min_distance:
                    min_distance = distance
                    min_distance_index = cluster_index

            #record or update cluster for each data points
            if cluster_affiliation[i] != min_distance_index:
               cluster_affiliation[i] = min_distance_index
                
        #recompute centroids
        for cluster_index in range(no_of_clusters):                                                  # Iterating over all centroid indexes
            grouped_points = data[cluster_affiliation == cluster_index]                              # The actual datapoints closest to the current centroid are extracted by using a boolean array 
            one_vector = np.ones(len(grouped_points))
            centroids[cluster_index] = (np.dot(one_vector, grouped_points) / len(grouped_points))    # The new centroid is now the mean of the grouped datapoints

        j = clustering_objective (data, centroids)
    
        print(f"New centroids in this iteration =\n{centroids}")
        print(f"|J - J_prev| = {abs(j - j_prev)}")
        print(divider)
        
        if abs(j - j_prev) <= 10**(-5):                                                              # Breaks the loop if there is no significant change in the value of J over an iteration
            break
        j_prev = j                                                                                   # Updating J_prev for later iterations.

    print(f"Final Centroids =\n{centroids}")
    print(f"Final Cluster Affiliations =\n{cluster_affiliation}")
    return centroids, cluster_affiliation


Driver funtion/Main Function

In [537]:
def create_3d_scatter_plot_of_dataset (data, class_labels):
    flower_names = class_labels.unique()
    symbols_dict = {
        flower_names[0] : "square-open",
        flower_names[1] : "diamond-open",
        flower_names[2] : "circle-open"
    }

    dataset_3d_plot = \
        px.scatter_3d (
            title      = "Dataset",
            data_frame = data,
            x          = data.columns[0],
            y          = data.columns[1],
            z          = data.columns[2],
            color      = class_labels,
            height     = 600
        )
    
    for i, d in enumerate(dataset_3d_plot.data):
        dataset_3d_plot.data[i].marker.symbol = symbols_dict[dataset_3d_plot.data[i].name]
    
    return dataset_3d_plot

In [538]:
def create_3d_scatter_plot_showing_cluster_affiliation (data, cluster_affiliation):
    clusters = np.unique(cluster_affiliation)
    symbols_dict = {
        clusters[0] : "square-open",
        clusters[1] : "diamond-open",
        clusters[2] : "circle-open"
    }
    
    data["Cluster Affiliation"] = cluster_affiliation

    dataset_3d_plot = \
        px.scatter_3d (
            title      = "Cluster Affiliation",
            data_frame = data,
            x          = data.columns[0],
            y          = data.columns[1],
            z          = data.columns[2],
            color      = data.columns[3],
            height     = 600
        )
    
    for i, d in enumerate(dataset_3d_plot.data):
        dataset_3d_plot.data[i].marker.symbol = symbols_dict[dataset_3d_plot.data[i].name]
        
    return dataset_3d_plot

In [539]:
def get_most_frequent (arr):
    return np.argmax(np.bincount(arr))

def create_3d_scatter_plot_showing_wrong_affiliation (data, cluster_affiliation, class_labels):
    data["Cluster Affiliation"] = cluster_affiliation
    data["Actual Class"] = class_labels
    most_common_affiliations = [get_most_frequent(cluster_affiliation[start : end]) for start, end in [(0, 50), (50, 100), (100, 150)]]
    
    affiliation_to_flower_name_dict ={
        most_common_affiliations[0]: class_labels[0],
        most_common_affiliations[1]: class_labels[50],
        most_common_affiliations[2]: class_labels[100]
    }

    data["Cluster Affiliation"] = data["Cluster Affiliation"].map(affiliation_to_flower_name_dict)
    data["Is Correct"] = data["Cluster Affiliation"] == data["Actual Class"]
    
    print("Data:")
    display(data)
    
    cluster_correctness_3d_scatter_plot = \
        px.scatter_3d (
            title      = "Cluster Correctness",
            data_frame = data,
            x          = data.columns[0],
            y          = data.columns[1],
            z          = data.columns[2],
            color      = "Is Correct",
            height     = 600
        )
    
    if cluster_correctness_3d_scatter_plot.data[0].name == "True":
        cluster_correctness_3d_scatter_plot.data[0].marker.color = "green"
    else:
        cluster_correctness_3d_scatter_plot.data[0].marker.color = "red"
        
    if cluster_correctness_3d_scatter_plot.data[1].name == "True":
        cluster_correctness_3d_scatter_plot.data[1].marker.color = "green"
    else:
        cluster_correctness_3d_scatter_plot.data[1].marker.color = "red"

    return cluster_correctness_3d_scatter_plot
    

In [540]:
def main():
    no_of_clusters = 3                                                                               # K clusters
    print(f"Number of clusters = {no_of_clusters}")
    
    dataset_path = 'iris/bezdekIris.data'
    
    data, class_label = load(dataset_path)
    print(divider)
    print(f"Data =\n{data}")
    print(f"Class Labels =\n{class_label}")
    print(divider)
    
    dimension_of_data = np.size(data, 1)                                                             # number of  data dimension in the data
    dimension_of_data = dimension_of_data - 1                                                        # We are using 3 dimensions
    
    print(f"Dimensions being used = {dimension_of_data}")
    print(divider)

    data_subset = data.iloc[:, : dimension_of_data]
    print(f"Data subset =\n{data_subset}")
    print(divider)
    
    centroids, cluster_affiliation = kmeans(data_subset, dimension_of_data, no_of_clusters, 300)
    dataset_3d_plot = create_3d_scatter_plot_of_dataset (data_subset, class_label)
    dataset_3d_plot.show()
    cluster_affiliation_3d_plot = create_3d_scatter_plot_showing_cluster_affiliation (data_subset, cluster_affiliation.astype(str))
    cluster_affiliation_3d_plot.show()
    cluster_correctness_3d_scatter_plot = create_3d_scatter_plot_showing_wrong_affiliation(data_subset, cluster_affiliation, class_label)
    cluster_correctness_3d_scatter_plot.show()
	
if __name__ == "__main__":
	main()
			

Number of clusters = 3
--------------------------------------------------------------------------------------
Data =
     Sepal Length  Sepal Width  Petal Length  Petal Width
0             5.1          3.5           1.4          0.2
1             4.9          3.0           1.4          0.2
2             4.7          3.2           1.3          0.2
3             4.6          3.1           1.5          0.2
4             5.0          3.6           1.4          0.2
5             5.4          3.9           1.7          0.4
6             4.6          3.4           1.4          0.3
7             5.0          3.4           1.5          0.2
8             4.4          2.9           1.4          0.2
9             4.9          3.1           1.5          0.1
10            5.4          3.7           1.5          0.2
11            4.8          3.4           1.6          0.2
12            4.8          3.0           1.4          0.1
13            4.3          3.0           1.1          0.1
14           

Data:


,Sepal Length,Sepal Width,Petal Length,Cluster Affiliation,Actual Class,Is Correct
0,5.1,3.5,1.4,Iris-setosa,Iris-setosa,True
1,4.9,3.0,1.4,Iris-setosa,Iris-setosa,True
2,4.7,3.2,1.3,Iris-setosa,Iris-setosa,True
3,4.6,3.1,1.5,Iris-setosa,Iris-setosa,True
4,5.0,3.6,1.4,Iris-setosa,Iris-setosa,True
5,5.4,3.9,1.7,Iris-setosa,Iris-setosa,True
6,4.6,3.4,1.4,Iris-setosa,Iris-setosa,True
7,5.0,3.4,1.5,Iris-setosa,Iris-setosa,True
8,4.4,2.9,1.4,Iris-setosa,Iris-setosa,True
9,4.9,3.1,1.5,Iris-setosa,Iris-setosa,True
